
# Part 2 : Cleaning the data

In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import regex as re

In [2]:
london_raw = pd.read_csv('london.csv')
mach_raw = pd.read_csv('manchester.csv')
birm_raw = pd.read_csv('birmingham.csv')
leeds_raw = pd.read_csv('leeds.csv')
glas_raw = pd.read_csv('glasgow.csv')
southamp_raw = pd.read_csv('southampton.csv')
livp_raw = pd.read_csv('liverpool.csv')
newcas_raw = pd.read_csv('newcastle.csv')

In [3]:
#list of all df's 
city_df_list = [london_raw, mach_raw, birm_raw, leeds_raw, glas_raw, southamp_raw, livp_raw, newcas_raw]
city_list = ['london','manchester','birmingham','leeds','glasgow','southampton','liverpool','newcastle']

In [4]:
for df,city in zip(city_df_list,city_list):
    df['city'] = '{}'.format(city)        

Combining all city dataframes together

In [5]:
df_raw = pd.concat(city_df_list, ignore_index = True)
df_raw.drop(columns=['Unnamed: 0'],inplace=True)
df_raw

,jobtitle,company,salary,location,city
0,Data Comms Engineer/ Data Cable Engineer,Installation Technology,£115 - £125 a day,London WC2R 2LS,london
1,Big Data Engineer,The App Experts,"£30,000 - £35,000 a year",London,london
2,Remote Data Scientist / Machine Learning Engin...,GCS Recruitment Specialists Ltd,"£40,000 - £90,000 a year",London,london
3,Data and Research Analyst,Cancer Research UK,"£31,000 a year",Stratford,london
4,Machine Learning Research Scientist,nPlan,None,London,london
...,...,...,...,...,...
5274,Junior DevOps Engineer,DXC,None,Newcastle upon Tyne,newcastle
5275,Change & Config Analyst,Northgate Public Services,None,Hartlepool TS24,newcastle
5276,Workforce Forecasting & Planning Analyst,Sitel,None,United Kingdom,newcastle
5277,Printed Circuit Board Design Engineer,TURBO POWER SYSTEMS,"£33,000 a year",Gateshead NE11 0QD,newcastle


### Cleaning salary data
Salary data is sometimes in form of a range, or string calculated hourly, daily, weekly or yearly. 
- Eliminate empty or 0 salary rows 
- Eliminate duplicate values
- Fix salaries that given as text or with ranges.
- Extract whether its short_term pay or long_term (define hour/day/week as short_term, month/year as long_term)


In [6]:
print("n. of entries:", len(df_raw))
df_raw = df_raw[df_raw['salary'] != 'None'].copy() #removing rows with empty salaries
df_raw = df_raw[~(df_raw.salary == 0)]
print("n. of entries after salary clean:",len(df_raw))
df_raw = df_raw[~df_raw.duplicated()] #removing duplicate rows with boolean mask
print("n. of entries after duplicate clean:",len(df_raw))
df_raw.reset_index(inplace=True,drop=True) #resetting index

n. of entries: 5279
n. of entries after salary clean: 2197
n. of entries after duplicate clean: 1691


In [7]:
#Get list of index / create boolean filter for job titles that do not match any in keyword list 
#I will only be looking at jobs with following keyword arguements. 

title_keyword_list = ['engineer','engineering','analyst','analysts','analytics','scientist','machine','data',
                      'artificial','intelligence','developer','insights', 'python','sql','software','cloud',
                      'architect', 'supply','database', 'ai','research','researcher','it','system','systems',
                      'product','mining']
mask_list = []
index_list = []
check_list = []  #the jobtitles that got cleaned
for index, title in enumerate(df_raw.jobtitle.str.lower().replace('(\W)',' ',regex=True)):
    counter_list = []
    for word in title.split():   #checks if any word in string matches the list of keywords
        if word in title_keyword_list:
            counter_list.append(True)  #adds to True to the counter if matches
        else:
            counter_list.append(False) #add to False to the counter if no match
    if sum(counter_list) > 0: #if any word matched (counter sum > 0) append True to mask list
        mask_list.append(True)
        index_list.append(index) #also append index to index list
    else:
        mask_list.append(False)  #if no word matched i.e sum(counter) = 0 append False 
        check_list.append(title)
        
print("index list length: ",len(index_list))
print("mask list length: ",len(mask_list))

df_raw = df_raw[mask_list].copy()  #applying boolean mask to df_raw
df_raw.reset_index(inplace=True,drop=True)
df_raw

index list length:  1392
mask list length:  1691


,jobtitle,company,salary,location,city
0,Data Comms Engineer/ Data Cable Engineer,Installation Technology,£115 - £125 a day,London WC2R 2LS,london
1,Big Data Engineer,The App Experts,"£30,000 - £35,000 a year",London,london
2,Remote Data Scientist / Machine Learning Engin...,GCS Recruitment Specialists Ltd,"£40,000 - £90,000 a year",London,london
3,Data and Research Analyst,Cancer Research UK,"£31,000 a year",Stratford,london
4,Data Scientist / Machine Learning Engineer,Blue Pelican,"£50,000 - £65,000 a year",London,london
...,...,...,...,...,...
1387,Senior Developer - IT,APCOA PARKING UK,"£40,000 a year",Newcastle upon Tyne,newcastle
1388,Infrstructure Engineer,NRG,"£30,000 - £40,000 a year",Cramlington,newcastle
1389,Data Developer - Text Mining/Artificial Intell...,Newcastle University,"£30,942 - £32,817 a year",Newcastle upon Tyne,newcastle
1390,Senior Product Manager,Maximus UK,"£54,000 - £64,000 a year",Newcastle upon Tyne,newcastle


Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary.

In [8]:
#creating function to extract hour/day/week/month/year
def salary_type(x):
    try:
        if 'hour' in x:
            return 'hourly','short_term'
        elif 'day' in x:
            return 'daily','short_term'
        elif 'week' in x:
            return 'weekly','short_term'        
        elif 'month' in x:
            return 'monthly','long_term'
        elif 'year' in x:
            return 'yearly','long_term'
        else:
            return 'None','None'
    except:
        pass
    
#creating new feature for type salary displayed as day, month or year

df_raw['salary_pay_type'] = df_raw.salary.map(lambda x: salary_type(x)[0]) #extract the salary_pay_type 
df_raw['salary_security'] = df_raw.salary.map(lambda x: salary_type(x)[1]) #extract the salary_security
print("n. of values with no salary description :",sum(df_raw.salary_pay_type == 'None'))

n. of values with no salary description : 0


In [9]:
#extract average from salary range
def extract_avg(salary):    
    salary = salary.replace(',','')
    if '.' in salary:
        salary = re.findall('\d+[.]\d+',salary)
    else:
        salary = re.findall('£(\d+,*\d+)|£(\d+)',salary)
        salary = [x for sublist in salary for x in sublist]
        while '' in salary:
            salary.remove('')
    if len(salary) > 1:
        salary = (sum([float(x) for x in salary])/2)
    else:
        salary = float(salary[0])
    return int(salary)

#convert salary into yearly salary taking into account salary ranges
def salary_conversion(x):
    if 'hour' in x:
        sal_hr = extract_avg(x)     #call extract_avg function to take care of ranges
        sal_yr = sal_hr*40*50       #hour payment: assume 40 hours a week, 50 weeks a year
        return int(sal_yr)
    elif 'day' in x:
        return extract_avg(x)*5*50  #day payment: assume 5 days a week, 50 weeks a year 
    elif 'week' in x:
        return extract_avg(x)*50    #week payment: assume 50 weeks a year    
    elif 'month' in x:
        return extract_avg(x)*11.5  #month payment: assume 11.5 months per year (~50 weeks)
    elif 'year' in x:
        return extract_avg(x)

df_raw.salary = df_raw.salary.apply(lambda x : salary_conversion(x))

- Feature extraction for job position level

In [10]:
#classified into 4 position levels (from indeed)
import re
entry = ['intern','internship','junior','jr','graduate','entry','apprentice','apprenticeship','trainee','entry']
senior = ['senior','sr','exec','executive','head','chief','officer','director','deputy','snr']
mid_level = ['adviser','advisor','lead','manager','principal','mid','higher','main','management','managing','advanced']
intermediate = [] #anything not included above

def get_position_level(string):
    x = re.sub('(\W)',' ',string.lower()).split()  #take string and replace all but (a-z, A-Z, 0-9), split into list
    if bool([True for word in entry if(word in x)]): #checks if any word in x list matches corresponding list above
        return 'entry'                               #gives True of False statement 
    elif bool([True for word in senior if(word in x)]):
        return 'senior'
    elif bool([True for word in mid_level if(word in x)]):
        return 'mid_level'
    else:
        return 'intermediate'

#add new position level feature 
df_raw['position_level'] = df_raw.jobtitle.apply(lambda x : get_position_level(x))

We want to predict a binary variable - whether the salary was low or high. 
- Use median salary reference point to convert salary into a binary variable.
- 'high' or 'low' if above or below median salary.

In [11]:
df_raw.salary.median()

38300.0

In [12]:
salary_median = df_raw.salary.median()
df_raw['salary_level'] = df_raw.salary.map(lambda x : 'high' if x >= salary_median else 'low' )

In [13]:
# save clean dataframe to csv
df_raw.to_csv('df_clean.csv')